# Clustering using convex hulls - high dimensional data

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.datasets import make_blobs
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from itertools import permutations
from scipy.spatial import ConvexHull
from quadprog import solve_qp
from sklearn.metrics import accuracy_score

## Create dataset

In [2]:
# Creating dataset 
centers = [[0.8, 0.5, 0.7, 0.6, 0.5, 0.5, 0.3, 0.3], 
           [0.2, 0.2, 0.2, 0.1, 0.1, 0.2, 0.3, 0.2], 
           [0.1, -0.1, -0.1, -0.2, -0.2, -0.1, 0, 0.1]]
stds = [0.4, 0.3, 0.4]   

X, labels_true = make_blobs(n_samples=1000, centers=centers, cluster_std=stds, random_state=0)        
point_indices = np.arange(1000)


## Get training and testing datasets

In [3]:
X_seeds, X_rest, y_seeds, y_rest, id_seeds, id_rest = train_test_split(X, labels_true, point_indices, test_size=0.33, random_state=42)


## Remap clustering result to original labels

In [4]:
def remap_labels(pred_labels, true_labels):
    """Rename prediction labels (clustered output) to best match true labels."""
    # from itertools import permutations # import this into script.
    pred_labels, true_labels = np.array(pred_labels), np.array(true_labels)
    assert pred_labels.ndim == 1 == true_labels.ndim
    assert len(pred_labels) == len(true_labels)
    cluster_names = np.unique(pred_labels)
    accuracy = 0

    perms = np.array(list(permutations(np.unique(true_labels))))

    remapped_labels = true_labels
    for perm in perms:
        flipped_labels = np.zeros(len(true_labels))
        for label_index, label in enumerate(cluster_names):
            flipped_labels[pred_labels == label] = perm[label_index]

        testAcc = np.sum(flipped_labels == true_labels) / len(true_labels)
        if testAcc > accuracy:
            accuracy = testAcc
            remapped_labels = flipped_labels

    return accuracy, remapped_labels

## Get initial clustering using K-means

In [5]:
kmeans = KMeans(n_clusters=3, random_state=9).fit(X_seeds)
initial_result = kmeans.labels_

intial_accuracy, remapped_initial_result = remap_labels(initial_result, y_seeds)
print(intial_accuracy)


0.8641791044776119


## Get convex hulls of the initial clustering

In [6]:
# Get the idices of the data points belonging to each cluster
indices = {}

for i in range(len(id_seeds)):
    if int(remapped_initial_result[i]) not in indices:
        indices[int(remapped_initial_result[i])] = [i]
    else:
        indices[int(remapped_initial_result[i])].append(i)

# Get convex hulls for each cluster
hulls = {}

for i in indices:
    hull = ConvexHull(X_seeds[indices[i]])
    hulls[i] = hull

print(hulls.keys())

dict_keys([2, 1, 0])


## Assign remaining points to the cluster of the closest convex hull

In [7]:
def point_in_hull(point, hull, tolerance=1e-12):
    return all(
        (np.dot(eq[:-1], point) + eq[-1] <= tolerance) for eq in hull.equations)

def proj2hull(z, equations):
    """
    Project `z` to the convex hull defined by the
    hyperplane equations of the facets

    Arguments
        z: array, shape (ndim,)
        equations: array shape (nfacets, ndim + 1)

    Returns
        x: array, shape (ndim,)
    """
    G = np.eye(len(z), dtype=float)
    a = np.array(z, dtype=float)
    C = np.array(-equations[:, :-1], dtype=float)
    b = np.array(equations[:, -1], dtype=float)
    x, f, xu, itr, lag, act = solve_qp(G, a, C.T, b, meq=0, factorized=True)
    return x

In [8]:
prediction = []

for z1 in X_rest:
    
    min_cluster_distance = 100000
    min_distance_point = ""
    min_cluster_distance_hull = ""
    
    for i in indices:

        p = proj2hull(z1, hulls[i].equations)

        dist = np.linalg.norm(z1-p)

        if dist < min_cluster_distance:
            min_cluster_distance = dist
            min_distance_point = p
            min_cluster_distance_hull = i
            
    prediction.append(min_cluster_distance_hull)

prediction = np.array(prediction)

## Evaluate final result

In [9]:
Y_pred = np.concatenate((remapped_initial_result, prediction))
Y_real = np.concatenate((y_seeds, y_rest))

final_accuracy, remapped_final_result = remap_labels(Y_pred, Y_real)

print("Accuracy of Convex Hull method:", final_accuracy)

Accuracy of Convex Hull method: 0.867


In [10]:
accuracy_score(Y_real, Y_pred)

0.867

## Run K-means on the entire dataset

In [11]:
kmeans = KMeans(n_clusters=3, random_state=9).fit(X)
result = kmeans.labels_

accuracy, remapped_result = remap_labels(result, labels_true)

print("Accuracy of K-means method:", accuracy)


Accuracy of K-means method: 0.866
